In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.transform
import skimage.filters
import skimage.feature
import skimage.restoration
from skimage.transform import resize
from skimage.filters import sobel
from skimage.feature import canny
from skimage.restoration import denoise_bilateral, denoise_tv_chambolle
from skimage.io import imread, imshow
from sklearn.cluster import KMeans
from scipy import misc
from skimage.color import rgb2gray
from skimage.io import imread, imshow
import shutil
import glob

import boto3
import sys
sys.path.insert(0,'/home/newton/images')
import os
import pickle

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
# tf.enable_eager_execution()

import tensorflow_hub as hub
# import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.experimental
import keras.applications.mobilenet
from keras.preprocessing import image
import file_image_processor

Using TensorFlow backend.


In [2]:
base_dir= '/home/newton/images/gender_images'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
batch_size = 100
IMG_W = 224
IMG_H = 224

In [3]:
image_gen_train = ImageDataGenerator(rescale=1/255)
train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=batch_size, 
                                                directory=train_dir, 
                                                shuffle=True, 
                                                target_size=(IMG_W,IMG_H),
                                                class_mode='sparse'
                                                )

Found 8000 images belonging to 2 classes.


In [4]:
image_gen_val = ImageDataGenerator(rescale=1/255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size, 
                                                 directory=val_dir, 
                                                 target_size=(IMG_W, IMG_H),
                                                 class_mode='sparse')

Found 2000 images belonging to 2 classes.


In [5]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"

feature_extractor = hub.KerasLayer(URL)

feature_extractor.trainable = False

m1_gender = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(5, activation='softmax')
])

In [13]:
train_data_gen[0][0].shape

(100, 224, 224, 3)

In [6]:
m1_gender.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
epochs = 5

history = m1_gender.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen
)

Epoch 1/5
80/80 [==============================] - 278s 3s/step - loss: 0.1659 - acc: 0.9352 - val_loss: 0.2009 - val_acc: 0.9270
Epoch 2/5
80/80 [==============================] - 277s 3s/step - loss: 0.1581 - acc: 0.9381 - val_loss: 0.2002 - val_acc: 0.9225
Epoch 3/5
80/80 [==============================] - 278s 3s/step - loss: 0.1574 - acc: 0.9374 - val_loss: 0.2110 - val_acc: 0.9165
Epoch 4/5
80/80 [==============================] - 276s 3s/step - loss: 0.1524 - acc: 0.9411 - val_loss: 0.1977 - val_acc: 0.9280
Epoch 5/5
80/80 [==============================] - 277s 3s/step - loss: 0.1466 - acc: 0.9438 - val_loss: 0.1956 - val_acc: 0.9260


In [9]:
m1_gender.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dense (Dense)                multiple                  6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [11]:
m1_gender.save('./models/gender_model_1.h5')

In [2]:
new_model = tf.keras.models.load_model('./models/gender_model_1.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [3]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dense (Dense)                multiple                  6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
# The image used here is female.

In [6]:
im = prepare_image(os.path.join('/home/newton/images/gender_images/train/female','008642.jpg'))

In [7]:
result = new_model.predict(im)

In [8]:
print (result)

[[9.9848807e-02 9.0014851e-01 2.1657875e-07 4.4057771e-07 2.0590005e-06]]


In [11]:
result[0][1]

0.9001485